## Ejercicio Web Scraping bs4/Selenium/Helium

_**url** = https://www.20minutos.es/_

Vamos a hacer Web Scraping de las primeras 3 paginas de las categorias de noticias: **Ciencia**, **Deporte**, **Gente**, **Economía**, **Grastronomía** y **Opinión**.

Y de cada noticia/articulo vamos a obtener:

- **Titulo**
- **Hora**
- **Fecha**
- **Autor**
- **Texto completo**
- **Categoria (columna objetivo)**

Genera un DataFrame con esta información y guardalo en el archivo **`20minutos_1.csv`**.

In [ ]:
import requests 

from bs4 import BeautifulSoup

from selenium import webdriver

from time import sleep

import helium

import pandas as pd
import numpy as np

In [7]:
# Driver de Chrome
chrome_driver = "chromedriver.exe"

In [8]:
browser = webdriver.Chrome(executable_path = chrome_driver) #Abre el navegador

browser.get("https://www.20minutos.es/") #Entra en 20minutos

browser.maximize_window() #Maximiza la ventana del navegador

In [9]:
#Ventana cookies
browser.find_element_by_id("didomi-notice-agree-button").click()
sleep(3)

In [10]:
#Acceso a categorías, selector CSS de "li" del elemento del menú "Más"
browser.find_element_by_css_selector("#ui-header-land > div.section-menu > nav > ul > li.primary-more.link.current.no-expandable.last").click()
sleep(3)

In [11]:
#Acceso a la categoría "Ciencia" por enlace
browser.find_element_by_link_text("Ciencia").click()
sleep(3)

In [12]:
#Extraemos el html y lo pasamos a objeto bs4
soup = BeautifulSoup(browser.page_source, "html.parser")
soup

<html lang="es-ES"><head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="width=1024" name="viewport"/>
<link href="https://imagenes.20minutos.es" rel="dns-prefetch"/><link href="https://sdk.privacy-center.org" rel="dns-prefetch"/><link href="https://sb.scorecardresearch.com" rel="dns-prefetch"/><link href="https://cdns.eu1.gigya.com" rel="dns-prefetch"/><link href="https://www.googletagmanager.com" rel="dns-prefetch"/><link href="https://www.googletagservices.com" rel="dns-prefetch"/><link href="https://hb.20m.es" rel="dns-prefetch"/><link href="https://securepubads.g.doubleclick.net" rel="dns-prefetch"/><link href="https://static.criteo.net" rel="dns-prefetch"/><link href="https://dkumiip2e9ary.cloudfront.net" rel="dns-prefetch"/><link href="https://dt15fd2xumytl.cloudfront.net" rel="dns-prefetch"/><link href="https://comentarios.20minutos.es" rel="dns-prefetch"/><link crossorigin="" href="https://fonts.gstatic.com/" rel="preconnect"/><link href="h

In [ ]:
lista_noticias_1 = []

#Bloque 1 de noticias (sólo en página 1)

for bs in soup.find("ul", class_ = "section-list").find_all("a")[1::2]:
    lista_noticias_1.append(bs["href"])

In [ ]:
lista_noticias_1

In [ ]:
#Bloque 2 de noticias (sólo en página 2)
#Lo identificamos con "media-content" que vale para todas las noticias
lista_enlaces = []

for bs in soup.find_all("div", class_ = "media-content"):
    
    enlace = bs.find("a")["href"]
    
    if enlace.startswith("https://www.20minutos.es/noticia/"):
        lista_enlaces.append(enlace)
        
len(lista_enlaces)

In [2]:
def noticiasCategoria(url_base, categoria, num_paginas):
    lista_urls = []
    
    #Recorremos las páginas
    for i in range(1, num_paginas + 1):
        
        #Elaboramos la página a la que hay que acceder
        url = f"{url_base}{categoria}/{i}/"
        print(url)
        
        #Obtener el response de la página con Helium sin abrir navegador
        browser = helium.start_chrome(url, headless = True)
        sleep(5)
        
        #Extraemos el html y lo pasamos a objeto bs4
        soup = BeautifulSoup(browser.page_source, "html.parser")
        
        #Recorremos el html para extraer las noticias
        for bs in soup.find_all("div", class_ = "media-content"):

            #Accedemos al enlace del atributo "href" de la etiqueta "a"
            enlace = bs.find("a")["href"]

            #Sólo incluimos en la lista si el enlace incluye "noticia"
            if (("noticia" in enlace) or (categoria in enlace)) and ("/imagenes/" not in enlace):
                lista_urls.append(enlace)
    
    
    return lista_urls

In [3]:
#Extracción de noticias de las 3 primeras páginas
resultado = noticiasCategoria("https://www.20minutos.es/", "ciencia", 3)
len(resultado)

https://www.20minutos.es/ciencia/1/
https://www.20minutos.es/ciencia/2/
https://www.20minutos.es/ciencia/3/


84

In [ ]:
%%time

categorias = ["ciencia", "deportes", "gente", "economia", "cultura", "opinion"]

dicc_enlaces = {}

num_paginas = 3

for categoria in categorias:
    
    dicc_enlaces[categoria] = noticiasCategoria("https://www.20minutos.es/", categoria, num_paginas)
    

In [ ]:
for categoria in categorias:
    print(f"{categoria}: ",len(dicc_enlaces[categoria]))

In [4]:
#df = pd.read_csv("urls_ejercicio1.csv")

#df.head()

,Categoría,Url
0,ciencia,https://www.20minutos.es/noticia/4958380/0/pen...
1,ciencia,https://www.20minutos.es/noticia/4958250/0/el-...
2,ciencia,https://www.20minutos.es/noticia/4958244/0/un-...
3,ciencia,https://www.20minutos.es/noticia/4958055/0/des...
4,ciencia,https://www.20minutos.es/noticia/4957924/0/un-...


In [ ]:
df = pd.DataFrame(dicc_enlaces.items(), columns = ["Categoría", "Url"])

In [ ]:
df = df.explode("Url").reset_index(drop = True)

In [5]:
df

,Categoría,Url
0,ciencia,https://www.20minutos.es/noticia/4958380/0/pen...
1,ciencia,https://www.20minutos.es/noticia/4958250/0/el-...
2,ciencia,https://www.20minutos.es/noticia/4958244/0/un-...
3,ciencia,https://www.20minutos.es/noticia/4958055/0/des...
4,ciencia,https://www.20minutos.es/noticia/4957924/0/un-...
...,...,...
510,opinion,https://www.20minutos.es/opinion/david-moreno-...
511,opinion,https://www.20minutos.es/opinion/carmelo-encin...
512,opinion,https://www.20minutos.es/opinion/el-eurotongo-...
513,opinion,https://www.20minutos.es/opinion/sereno-joan-f...


In [ ]:
df.to_csv("urls_ejercicio1.csv", index = False, sep = ",")

In [ ]:
#Recorremos el dataframe para extraer información de las noticias
url = df["Url"][0]
# url
browser = helium.start_chrome(url, headless = True)
sleep(1)

soup = BeautifulSoup(browser.page_source, "html.parser")
soup

In [ ]:
soup.find("h1", class_ = "notice-title").text.strip()

In [ ]:
soup.find("span", class_ = "article-date").text.split("-")[1].strip()

In [ ]:
soup.find("span", class_ = "article-date").text.split("-")[0].strip()

In [ ]:
soup.find("span", class_ = "article-author").text

In [ ]:
parrafos = list()

for bs in soup.find_all("p", class_ = "paragraph"):
    parrafos.append(bs.text.strip())
    
parrafos = "\n".join(parrafos)

print(parrafos)

In [6]:
def info_articulos(url):
    browser = helium.start_chrome(url, headless = True)
    sleep(1)
    
    soup = BeautifulSoup(browser.page_source, "html.parser")
    
    # Titulo
    try:
        titulo = soup.find("h1", class_ = "notice-title").text.strip()
    except:
        titulo = np.nan
        
    # Hora
    try:
        hora = soup.find("span", class_ = "article-date").text.split("-")[1].strip()
    except:
        hora = np.nan
        
    # Fecha
    try:
        fecha = soup.find("span", class_ = "article-date").text.split("-")[0].strip()
    except:
        fecha = np.nan
        
    # Autor
    try:
        autor = soup.find("span", class_ = "article-author").text
    except:
        autor = np.nan
        
    # Texto
    try:
        texto = list()

        for bs in soup.find_all("p", class_ = "paragraph"):
            texto.append(bs.text.strip())

        texto = "\n".join(texto)

    except:
        texto = np.nan
        
    return [titulo, hora, fecha, autor, texto]

In [ ]:
datos = list()

for indice, url in enumerate(df["Url"]):
    print(indice)
    datos.append(info_articulos(url))
    
datos

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28


In [ ]:
df_info = pd.DataFrame(datos, columns = ["titulo", "hora", "fecha", "autor", "texto"])

In [ ]:
pd.concat([df, df_info], axis = 1)

In [ ]:
df = pd.concat([df, df_info], axis = 1)

df.to_csv("20minutos_1.csv", index = False, sep = ",")

- **Escribe el código para sacar la información de las primeras 5 páginas de todas las categorias (23 categorias). Guarda esta información en el archivo `20minutos_2.csv`**

In [ ]:
%%time

categorias = ["ciencia", "deportes", "gente", "economia", "cultura", "opinion", "andalucia",
              "comunidad-valenciana", "cinemania", "salud", "tecnologia"]

dicc_enlaces = {}

num_paginas = 5

for categoria in categorias:
    
    dicc_enlaces[categoria] = noticiasCategoria("https://www.20minutos.es/", categoria, num_paginas)
    
    
df = pd.DataFrame(dicc_enlaces.items(), columns = ["Categoría", "Url"])   

df = df.explode("Url")


datos = list()

for indice, url in enumerate(df["Url"]):
    print(indice)
    datos.append(info_articulos(url))
    
df_info = pd.DataFrame(datos, columns = ["titulo", "hora", "fecha", "autor", "texto"])

df = pd.concat([df, df_info], axis = 1)

df.to_csv("20minutos_2.csv", index = False, sep = ",")

- **Escribe el código para sacar la información de todas las categorias del mes de enero. Guarda esta información en el archivo `20minutos_3.csv`.**